In [ ]:
import re

def get_ip(dom):
    # e.g. ※ 發信站: 批踢踢實業坊(ptt.cc), 來自: 122.96.47.157 (中國)
    pattern = '來自: \d+\.\d+\.\d+\.\d+'
    match = re.search(pattern.dom)
    if match:
        return match.group(0).replace('來自: ', '')
    else:
        return None
    